<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Encode_and_Decode_TinyURLhash_product_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import threading
import string

# Simplified Snowflake ID Generator
class Snowflake:
    def __init__(self, datacenter_id: int, worker_id: int):
        self.datacenter_id = datacenter_id
        self.worker_id = worker_id
        self.sequence = 0
        self.last_timestamp = -1

        # Constants for bit lengths
        self.worker_id_bits = 5
        self.datacenter_id_bits = 5
        self.sequence_bits = 12

        self.max_worker_id = -1 ^ (-1 << self.worker_id_bits)
        self.max_datacenter_id = -1 ^ (-1 << self.datacenter_id_bits)

        # Shifts
        self.worker_id_shift = self.sequence_bits
        self.datacenter_id_shift = self.sequence_bits + self.worker_id_bits
        self.timestamp_left_shift = self.sequence_bits + self.worker_id_bits + self.datacenter_id_bits
        self.sequence_mask = -1 ^ (-1 << self.sequence_bits)

        # Custom epoch (Twitter's epoch is used in their original Snowflake algorithm)
        self.epoch = 1288834974657

        self.lock = threading.Lock()

    def _time_gen(self) -> int:
        return int(time.time() * 1000)

    def _til_next_millis(self, last_timestamp: int) -> int:
        timestamp = self._time_gen()
        while timestamp <= last_timestamp:
            timestamp = self._time_gen()
        return timestamp

    def get_id(self) -> int:
        with self.lock:
            timestamp = self._time_gen()
            if timestamp < self.last_timestamp:
                raise Exception("Clock moved backwards. Refusing to generate id.")

            if self.last_timestamp == timestamp:
                self.sequence = (self.sequence + 1) & self.sequence_mask
                if self.sequence == 0:
                    timestamp = self._til_next_millis(self.last_timestamp)
            else:
                self.sequence = 0

            self.last_timestamp = timestamp
            # Compose the 64-bit ID
            unique_id = ((timestamp - self.epoch) << self.timestamp_left_shift) | \
                        (self.datacenter_id << self.datacenter_id_shift) | \
                        (self.worker_id << self.worker_id_shift) | \
                        self.sequence
            return unique_id

# Base62 encoding function
def base62_encode(num: int) -> str:
    characters = string.digits + string.ascii_letters  # 0-9, a-z, A-Z
    if num == 0:
        return characters[0]
    base = len(characters)
    result = []
    while num:
        num, rem = divmod(num, base)
        result.append(characters[rem])
    return ''.join(reversed(result))

# Production-ready TinyURL Service
class TinyURLService:
    def __init__(self, datacenter_id: int = 0, worker_id: int = 0):
        # Initialize the Snowflake generator for unique IDs.
        self.snowflake = Snowflake(datacenter_id, worker_id)
        # In production, this mapping would be stored in a persistent database.
        self.urlMap = {}
        self.baseUrl = "http://tinyurl.com/"

    def encode(self, longUrl: str) -> str:
        """
        Encodes a long URL to a tiny URL.
        """
        # Generate a unique ID using Snowflake algorithm.
        unique_id = self.snowflake.get_id()
        # Convert the numeric ID to a Base62 string.
        key = base62_encode(unique_id)
        # Store the mapping. In production, use a persistent datastore.
        self.urlMap[key] = longUrl
        # Return the full short URL.
        return self.baseUrl + key

    def decode(self, shortUrl: str) -> str:
        """
        Decodes a tiny URL back to the original long URL.
        """
        # Extract the key by removing the base URL.
        key = shortUrl.replace(self.baseUrl, "")
        # Return the original URL, or an empty string if not found.
        return self.urlMap.get(key, "")

# Example usage:
if __name__ == "__main__":
    service = TinyURLService(datacenter_id=1, worker_id=1)

    long_url = "https://leetcode.com/problems/design-tinyurl"
    short_url = service.encode(long_url)
    print("Short URL:", short_url)

    original_url = service.decode(short_url)
    print("Decoded URL:", original_url)
